<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a custom object detection model with TensorFlow Lite Model Maker

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 577 kB 5.0 MB/s 
     |████████████████████████████████| 840 kB 63.5 MB/s 
     |████████████████████████████████| 1.1 MB 37.2 MB/s 
     |████████████████████████████████| 238 kB 62.8 MB/s 
     |████████████████████████████████| 60.9 MB 1.2 MB/s 
     |████████████████████████████████| 77 kB 2.4 MB/s 
     |████████████████████████████████| 10.9 MB 38.3 MB/s 
     |████████████████████████████████| 3.4 MB 32.1 MB/s 
     |████████████████████████████████| 87 kB 7.9 MB/s 
     |████████████████████████████████| 1.3 MB 55.5 MB/s 
     |████████████████████████████████| 128 kB 73.2 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 497.9 MB 4.2 kB/s 
     |████████████████████████████████| 352 kB 71.9 MB/s 
     |████████████████████████████████| 462 kB 64.6 MB/s 
     |████████████████████████████████| 1.4 MB 58.4 MB/s 
     |████████████████████████████████| 5.8 MB 52.2 MB/s 
     |███████████

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare the dataset

This dataset contains about 1000 images of 2 types of Waste: a recyclable waste and a Non-recyclable waste.

We start with downloading the dataset.

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="HekdMCnHKWWTKkVpmYh2")
project = rf.workspace("sukkur-iba-university-dxckm").project("waste_data_6")
dataset = project.version(1).download("voc")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 546 kB/s 
     |████████████████████████████████| 67 kB 3.6 MB/s 
     |████████████████████████████████| 178 kB 34.1 MB/s 
     |████████████████████████████████| 145 kB 60.5 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 138 kB 57.2 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=ab5add7baf4a6c88c052cc9c35fa6960586b516ed146c4d3f7897a8ad4494e87
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certi

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Waste_data_6-1 in voc:: 100%|██████████| 1877/1877 [00:01<00:00, 1534.01it/s]


## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/Waste_data_6-1/valid',
    '/content/Waste_data_6-1/valid',
    ['R', 'NR']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/Waste_data_6-1/train',
    '/content/Waste_data_6-1/train',
    ['R', 'NR']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
186/186 [==============================] - 85s 248ms/step - det_loss: 1.0734 - cls_loss: 0.6388 - box_loss: 0.0087 - reg_l2_loss: 0.0631 - loss: 1.1366 - learning_rate: 0.0065 - gradient_norm: 5.0296 - val_det_loss: 1.0846 - val_cls_loss: 0.7647 - val_box_loss: 0.0064 - val_reg_l2_loss: 0.0634 - val_loss: 1.1480
Epoch 2/20
186/186 [==============================] - 44s 234ms/step - det_loss: 0.5979 - cls_loss: 0.3519 - box_loss: 0.0049 - reg_l2_loss: 0.0635 - loss: 0.6614 - learning_rate: 0.0049 - gradient_norm: 4.9610 - val_det_loss: 0.8072 - val_cls_loss: 0.4663 - val_box_loss: 0.0068 - val_reg_l2_loss: 0.0636 - val_loss: 0.8708
Epoch 3/20
186/186 [==============================] - 46s 249ms/step - det_loss: 0.5293 - cls_loss: 0.3198 - box_loss: 0.0042 - reg_l2_loss: 0.0637 - loss: 0.5930 - learning_rate: 0.0048 - gradient_norm: 4.8672 - val_det_loss: 0.7475 - val_cls_loss: 0.4313 - val_box_loss: 0.0063 - val_reg_l2_loss: 0.0638 - val_loss: 0.8113
Epoch 4/20
186/186 [=====

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

12/12 [==============================] - 29s 1s/step



{'AP': 0.7263084,
 'AP50': 0.9974865,
 'AP75': 0.93036586,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.72634184,
 'ARmax1': 0.7619148,
 'ARmax10': 0.8024353,
 'ARmax100': 0.80339754,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.80339754,
 'AP_/R': 0.701551,
 'AP_/NR': 0.75106585}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
model.export(export_dir='.', tflite_filename='Waste_Sorter_Final.tflite')

### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('Waste_Sorter_Final.tflite', val_data)

  4/745 [..............................] - ETA: 30:08

KeyboardInterrupt: ignored

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('Waste_Sorter_Final.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_data = object_detector.DataLoader.from_pascal_voc(
    '/content/Waste_data_6-1/test',
    '/content/Waste_data_6-1/test',
    ['R', 'NR']
)

In [ ]:
model.evaluate_tflite('Waste_Sorter_Final.tflite', test_data)

51/51 [==============================] - 127s 2s/step



{'AP': 0.58290285,
 'AP50': 1.0,
 'AP75': 0.4734074,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.58290285,
 'ARmax1': 0.602,
 'ARmax10': 0.609,
 'ARmax100': 0.609,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.609,
 'AP_/R': 0.45049506,
 'AP_/NR': 0.7153107}